In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
df_f_dm =  pd.read_csv("Forecast_demand.csv",parse_dates=["date"])
df_f_r = pd.read_csv("Roof_Forecast.csv",parse_dates=["date"])
df_a_dm = pd.read_csv("Actual_Demand.csv",parse_dates=["date"])
df_a_r = pd.read_csv("Actual_Roof.csv",parse_dates=["date"])
df_train = pd.read_csv("new_train.csv",parse_dates=["date"])

- Align the time, Check the starting/end date
- Giving each forecast period special name, ex:operational 0.5 hour 
- integrate df_train
- solving last changed update time in 5 min interval


In [3]:
dataframes = {
    'Forecast Demand': df_f_dm,
    'Forecast Roof': df_f_r,
    'Actual Demand': df_a_dm,
    'Actual Roof': df_a_r,
    "Train":df_train
}

for name, df in dataframes.items():
    print(f"{name}:")
    print(f"  Start: {df['date'].min()}")
    print(f"  End: {df['date'].max()}")
    print(f"  Total days: {(df['date'].max() - df['date'].min()).days + 1}")
    print()

Forecast Demand:
  Start: 2023-08-26 23:30:00
  End: 2024-09-28 23:00:00
  Total days: 399

Forecast Roof:
  Start: 2023-08-31 00:00:00
  End: 2024-09-25 23:30:00
  Total days: 392

Actual Demand:
  Start: 2023-08-27 00:00:00
  End: 2024-09-28 23:30:00
  Total days: 399

Actual Roof:
  Start: 2023-08-30 23:30:00
  End: 2024-09-25 23:00:00
  Total days: 392

Train:
  Start: 2023-07-20 00:05:00
  End: 2024-10-01 00:00:00
  Total days: 439



In [4]:
start_date = max(df['date'].min() for df in dataframes.values())
end_date = min(df['date'].max() for df in dataframes.values())

print(f"Common date range:")
print(f"  Start: {start_date}")
print(f"  End: {end_date}")
print(f"  Total days: {(end_date - start_date).days + 1}")

Common date range:
  Start: 2023-08-31 00:00:00
  End: 2024-09-25 23:00:00
  Total days: 392


In [5]:
trimmed_dataframes = {}
for name, df in dataframes.items():
    trimmed_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    trimmed_dataframes[name] = trimmed_df
    print(f"{name} trimmed shape: {trimmed_df.shape}")

Forecast Demand trimmed shape: (94055, 21)
Forecast Roof trimmed shape: (94070, 22)
Actual Demand trimmed shape: (94075, 3)
Actual Roof trimmed shape: (94058, 4)
Train trimmed shape: (112885, 3)


In [6]:
for name, df in trimmed_dataframes.items():
    print(f"{name}:")
    print(f"  Start: {df['date'].min()}")
    print(f"  End: {df['date'].max()}")
    print(f"  Total days: {(df['date'].max() - df['date'].min()).days + 1}")
    print()

Forecast Demand:
  Start: 2023-08-31 00:00:00
  End: 2024-09-25 23:00:00
  Total days: 392

Forecast Roof:
  Start: 2023-08-31 00:00:00
  End: 2024-09-25 23:00:00
  Total days: 392

Actual Demand:
  Start: 2023-08-31 00:00:00
  End: 2024-09-25 23:00:00
  Total days: 392

Actual Roof:
  Start: 2023-08-31 00:00:00
  End: 2024-09-25 23:00:00
  Total days: 392

Train:
  Start: 2023-08-31 00:00:00
  End: 2024-09-25 23:00:00
  Total days: 392



In [8]:
for name, df in trimmed_dataframes.items():
    print(name)
    print(len(df))
    df.to_csv(f"./Trim/{name}.csv",index=False)
    

Forecast Demand
94055
Forecast Roof
94070
Actual Demand
94075
Actual Roof
94058
Train
112885


In [9]:
for name, df in trimmed_dataframes.items():
    if 'Region' in df.columns:
        duplicates = df[df.duplicated(subset=['date', 'Region'], keep=False)]
        print(f"{name}:")
        print(f"  Total line: {len(df)}")
        
        print(duplicates)
       
        print()

Forecast Demand:
  Total line: 94055
                     date Region  0.5 hour future  1 hour future  \
16725 2023-11-04 16:00:00   NSW1           7298.0         7523.0   
16726 2023-11-04 16:00:00   QLD1           6084.0         6386.0   
16727 2023-11-04 16:00:00   VIC1           3450.0         3834.0   
16728 2023-11-04 16:00:00   TAS1           1180.0         1211.0   
16729 2023-11-04 16:00:00    SA1            440.0          566.0   
16730 2023-11-04 16:00:00   NSW1           7523.0         7652.0   
16731 2023-11-04 16:00:00   QLD1           6386.0         6612.0   
16732 2023-11-04 16:00:00   VIC1           3834.0         4224.0   
16733 2023-11-04 16:00:00   TAS1           1211.0         1228.0   
16734 2023-11-04 16:00:00    SA1            566.0          823.0   
37565 2024-01-30 13:00:00   NSW1           8485.0         8815.0   
37566 2024-01-30 13:00:00   QLD1           8072.0         8234.0   
37567 2024-01-30 13:00:00   VIC1           3661.0         3713.0   
37568 2024-

### Duplicate data point in Demand Forecast
- 28485 2023-11-04 16:00:00
- 49326 2024-01-30 13:00:00 
- 84609 2024-06-25 13:00:00 

### Duplicate data point in Forecast Roof 

- 2024-09-05 13:00:00


## Import Trim df again

In [39]:
df_f_dm =  pd.read_csv("./Trim/Forecast Demand.csv",parse_dates=["date"])
df_f_r = pd.read_csv("./Trim/Forecast Roof.csv",parse_dates=["date"])
df_a_dm = pd.read_csv("./Trim/Actual Demand.csv",parse_dates=["date"])
df_a_r = pd.read_csv("./Trim/Actual Roof.csv",parse_dates=["date"])
df_train = pd.read_csv("./Trim/Train.csv",parse_dates=["date"])

new_dfs = {
    'Forecast Demand': df_f_dm,
    'Forecast Roof': df_f_r,
    'Actual Demand': df_a_dm,
    'Actual Roof': df_a_r,
    "Train":df_train
}

In [40]:
import pandas as pd
from datetime import timedelta

def check_dataframes(dfs):
    
    all_regions = ['NSW1', 'QLD1', 'VIC1', 'SA1', 'TAS1']

   
    all_dates = [df['date'] for df in dfs.values()]
    min_date = min([dates.min() for dates in all_dates])
    max_date = max([dates.max() for dates in all_dates])

    
    full_date_range = pd.date_range(start=min_date, end=max_date, freq='30T')

    for name, df in dfs.items():
        print(f"Check {name}:")
        
        # Check missing eate
        missing_dates = set(full_date_range) - set(df['date'])
        if missing_dates:
            print(f"  缺失的日期:")
            for date in sorted(missing_dates):
                print(f"    {date}")

        # ckeck region
        if name == "Train":
            continue
        grouped = df.groupby('date')
        for date, group in grouped:
            missing_regions = set(all_regions) - set(group['Region'])
            if missing_regions:
                print(f"  日期 {date} 缺失的地區:")
                for region in missing_regions:
                    print(f"    {region}")

        print()  


check_dataframes(new_dfs)

Check Forecast Demand:
  缺失的日期:
    2023-12-05 13:30:00
    2023-12-28 16:00:00
    2024-09-05 13:30:00
    2024-09-05 14:00:00

Check Forecast Roof:
  缺失的日期:
    2023-12-05 14:00:00

Check Actual Demand:

Check Actual Roof:
  缺失的日期:
    2023-12-05 13:00:00
    2023-12-05 13:30:00
    2024-09-05 14:00:00
  日期 2024-05-28 11:00:00 缺失的地區:
    SA1
  日期 2024-05-28 11:30:00 缺失的地區:
    SA1

Check Train:


- 檢查 Forecast Demand:
  缺失的日期:
    2023-12-05 13:30:00
    2023-12-28 16:00:00

- 檢查 Forecast Roof:
  缺失的日期:
    2023-12-05 14:00:00

- 檢查 Actual Demand:

- 檢查 Actual Roof:
  缺失的日期:
    2023-12-05 13:00:00
    2023-12-05 13:30:00
  日期 2024-05-28 11:00:00 缺失的地區:
    SA1
  日期 2024-05-28 11:30:00 缺失的地區:
    SA1

- 檢查 Train:

In [41]:
import pandas as pd
import numpy as np

def fill_forecast_data(df, missing_time):
    
    # df['date'] = pd.to_datetime(df['date'])
    
    
    prev_time = missing_time - pd.Timedelta(minutes=30)
    next_time = missing_time + pd.Timedelta(minutes=30)
    
   
    prev_data = df[df['date'] == prev_time]
    
   
    next_data = df[df['date'] == next_time]
    

    new_rows = []
    
    for region in df['Region'].unique():
        prev_row = prev_data[prev_data['Region'] == region].iloc[0]
        next_row = next_data[next_data['Region'] == region].iloc[0]
        
        new_row = {
            'date': missing_time,
            'Region': region
        }
        

        forecast_columns = [col for col in df.columns if 'future' in col]
        
        
        for i, col in enumerate(forecast_columns[:-1]): 
            next_col = forecast_columns[i+1]
            new_row[col] = prev_row[next_col]
        
        # For 9.5 hours future），use next row's 9 hours future
        last_col = forecast_columns[-1]
        second_last_col = forecast_columns[-2]
        new_row[last_col] = next_row[second_last_col]
        
        new_rows.append(new_row)
    

    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    

    df = df.sort_values(['date', 'Region'])
    
    return df


forecast_demand = new_dfs['Forecast Demand']
forecast_demand = fill_forecast_data(forecast_demand, pd.Timestamp('2023-12-05 13:30:00'))
forecast_demand = fill_forecast_data(forecast_demand, pd.Timestamp('2023-12-28 16:00:00'))




forecast_roof = new_dfs['Forecast Roof']
forecast_roof = fill_forecast_data(forecast_roof, pd.Timestamp('2023-12-05 14:00:00'))


new_dfs['Forecast Demand'] = forecast_demand
new_dfs['Forecast Roof'] = forecast_roof


check_dataframes(new_dfs)

Check Forecast Demand:
  缺失的日期:
    2024-09-05 13:30:00
    2024-09-05 14:00:00

Check Forecast Roof:

Check Actual Demand:

Check Actual Roof:
  缺失的日期:
    2023-12-05 13:00:00
    2023-12-05 13:30:00
    2024-09-05 14:00:00
  日期 2024-05-28 11:00:00 缺失的地區:
    SA1
  日期 2024-05-28 11:30:00 缺失的地區:
    SA1

Check Train:


In [42]:
import pandas as pd
import numpy as np

def fill_missing_data(df):

    # df['date'] = pd.to_datetime(df['date'])
    
    
    all_dates = pd.date_range(df['date'].min(), df['date'].max(), freq='30T')
    all_regions = df['Region'].unique()
    

    full_df = pd.DataFrame([(date, region) for date in all_dates for region in all_regions],
                           columns=['date', 'Region'])
    

    merged_df = pd.merge(full_df, df, on=['date', 'Region'], how='left')
    

    filled_data = []
    
    # Deal every region
    for region in all_regions:
        region_data = merged_df[merged_df['Region'] == region].sort_values('date')
        
        i = 0
        while i < len(region_data):
            row = region_data.iloc[i]
            if pd.isnull(row['OPOWER']):
                # For continues loss
                j = i
                while j < len(region_data) and pd.isnull(region_data.iloc[j]['OPOWER']):
                    j += 1
                
                if i > 0 and j < len(region_data):
                    prev_value = region_data.iloc[i-1]['OPOWER']
                    next_value = region_data.iloc[j]['OPOWER']
                    gap = j - i + 1
                    
                    for k in range(i, j):
                        interpolated_row = region_data.iloc[k].copy()
                        interpolated_row['OPOWER'] = prev_value + (next_value - prev_value) * (k - i + 1) / gap
                        if pd.isnull(interpolated_row['LASTCHANGED']):
                            interpolated_row['LASTCHANGED'] = interpolated_row['date']
                        filled_data.append(interpolated_row)
                    
                    i = j  
                else:
                    filled_data.append(row)
                    i += 1
            else:
                filled_data.append(row)
                i += 1
    

    new_df = pd.DataFrame(filled_data)
    

    new_df = new_df.dropna()
    

    new_df = new_df.sort_values(['date', 'Region'])
    
    return new_df


filled_df = fill_missing_data(new_dfs['Actual Roof'])

new_dfs['Actual Roof'] = filled_df


print(filled_df[(filled_df['date'] == '2023-12-05 13:00:00') | 
                (filled_df['date'] == '2023-12-05 13:30:00') |
                (filled_df['date'] == '2024-05-28 11:00:00') |
                (filled_df['date'] == '2024-05-28 11:30:00')])

                     date Region       OPOWER          LASTCHANGED
23170 2023-12-05 13:00:00   NSW1  4287.297000  2023-12-05 13:00:00
23171 2023-12-05 13:00:00   QLD1  3493.172333  2023-12-05 13:00:00
23172 2023-12-05 13:00:00    SA1   985.937667  2023-12-05 13:00:00
23173 2023-12-05 13:00:00   TAS1   186.708667  2023-12-05 13:00:00
23174 2023-12-05 13:00:00   VIC1  2477.908667  2023-12-05 13:00:00
23175 2023-12-05 13:30:00   NSW1  4162.728000  2023-12-05 13:30:00
23176 2023-12-05 13:30:00   QLD1  3414.620667  2023-12-05 13:30:00
23177 2023-12-05 13:30:00    SA1  1055.488333  2023-12-05 13:30:00
23178 2023-12-05 13:30:00   TAS1   194.098333  2023-12-05 13:30:00
23179 2023-12-05 13:30:00   VIC1  2301.208333  2023-12-05 13:30:00
65150 2024-05-28 11:00:00   NSW1  2751.415000  2024/05/28 11:19:02
65151 2024-05-28 11:00:00   QLD1  1507.219000  2024/05/28 11:19:02
65152 2024-05-28 11:00:00    SA1   709.188667  2024-05-28 11:00:00
65153 2024-05-28 11:00:00   TAS1    61.121000  2024/05/28 11:1

In [43]:
check_dataframes(new_dfs)

Check Forecast Demand:
  缺失的日期:
    2024-09-05 13:30:00
    2024-09-05 14:00:00

Check Forecast Roof:

Check Actual Demand:

Check Actual Roof:

Check Train:


In [44]:
def fill_2_forecast_data(df, missing_time):
    prev_time = missing_time - pd.Timedelta(minutes=30)  # 前一個時間點
    
    prev_data = df[df['date'] == prev_time]  # 獲取前一個時間點的數據

    new_rows = []

    for region in df['Region'].unique():
        
        prev_row = prev_data[prev_data['Region'] == region].iloc[0]
        
        new_row = {
            'date': missing_time,
            'Region': region
        }
        
        
        forecast_columns = [col for col in df.columns if 'future' in col]
        
        
        if missing_time == pd.Timestamp('2024-09-05 13:30:00'):
            
            # 13:30 缺失，用 13:00 的 forecast 1~9.5 填補 forecast 0.5~9
            for i, col in enumerate(forecast_columns[:-1]):  # 從 0.5 到 9
                prev_col = forecast_columns[i + 1]  # 使用前一列 (1 到 9.5)
                new_row[col] = prev_row[prev_col]
            # 9.5 小時的值設置與 9 小時一致
            new_row[forecast_columns[-1]] = new_row[forecast_columns[-2]]
            
        elif missing_time == pd.Timestamp('2024-09-05 14:00:00'):
           
            # 14:00 缺失，用 13:00 的 forecast 1.5~9.5 填補 forecast 0.5~8.5
            for i, col in enumerate(forecast_columns[:-2]):  # 從 0.5 到 8.5
                
                prev_col = forecast_columns[i + 2]  # 使用 1.5 到 9.5
                new_row[col] = prev_row[prev_col]
            # 9 和 9.5 小時的值設置與 8.5 小時一致
            new_row[forecast_columns[-2]] = new_row[forecast_columns[-3]]  # 9 = 8.5
            new_row[forecast_columns[-1]] = new_row[forecast_columns[-3]]  # 9.5 = 8.5
            
        
        new_rows.append(new_row)

    # 將新行添加到原始 DataFrame 中
    

    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    df = df.sort_values(['date', 'Region'])  # 排序

    return df


In [45]:
forecast_demand = new_dfs['Forecast Demand']
forecast_demand = fill_2_forecast_data(forecast_demand, pd.Timestamp('2024-09-05 13:30:00'))
forecast_demand = fill_2_forecast_data(forecast_demand, pd.Timestamp('2024-09-05 14:00:00'))
new_dfs['Forecast Demand'] = forecast_demand
check_dataframes(new_dfs)

Check Forecast Demand:

Check Forecast Roof:

Check Actual Demand:

Check Actual Roof:

Check Train:


In [46]:
for name,df in new_dfs.items():
    print(name)
    # print(df.head())
    # print(df.tail())
    print(len(df))
    print(df.columns)

Forecast Demand
94075
Index(['date', 'Region', '0.5 hour future', '1 hour future',
       '1.5 hours future', '2 hours future', '2.5 hours future',
       '3 hours future', '3.5 hours future', '4 hours future',
       '4.5 hours future', '5 hours future', '5.5 hours future',
       '6 hours future', '6.5 hours future', '7 hours future',
       '7.5 hours future', '8 hours future', '8.5 hours future',
       '9 hours future', '9.5 hours future'],
      dtype='object')
Forecast Roof
94075
Index(['date', 'Region', '0.5 hour future', '1 hour future',
       '1.5 hours future', '2 hours future', '2.5 hours future',
       '3 hours future', '3.5 hours future', '4 hours future',
       '4.5 hours future', '5 hours future', '5.5 hours future',
       '6 hours future', '6.5 hours future', '7 hours future',
       '7.5 hours future', '8 hours future', '8.5 hours future',
       '9 hours future', '9.5 hours future', '10 hours future'],
      dtype='object')
Actual Demand
94075
Index(['date', 'Reg

### Combine Actual/Forecast Demand/Roof together

In [47]:
import pandas as pd


for name, df in new_dfs.items():
    prefix = f"{name.replace(' ', '_')}_"
    df.columns = [prefix + col if col not in ['date', 'Region'] else col for col in df.columns]


merged_df = new_dfs['Forecast Demand']

for name in ['Forecast Roof', 'Actual Demand', 'Actual Roof']:
    merged_df = pd.merge(merged_df, new_dfs[name], 
                         on=['date', 'Region'], 
                         how='outer')


merged_df = merged_df.sort_values(['date', 'Region'])


merged_df = merged_df.reset_index(drop=True)


print(f"rows: {len(merged_df)}")
print(f"cols: {merged_df.shape[1]}")
print(merged_df.columns.tolist())


merged_df.to_csv('merged_data.csv', index=False)


rows: 94075
cols: 44
['date', 'Region', 'Forecast_Demand_0.5 hour future', 'Forecast_Demand_1 hour future', 'Forecast_Demand_1.5 hours future', 'Forecast_Demand_2 hours future', 'Forecast_Demand_2.5 hours future', 'Forecast_Demand_3 hours future', 'Forecast_Demand_3.5 hours future', 'Forecast_Demand_4 hours future', 'Forecast_Demand_4.5 hours future', 'Forecast_Demand_5 hours future', 'Forecast_Demand_5.5 hours future', 'Forecast_Demand_6 hours future', 'Forecast_Demand_6.5 hours future', 'Forecast_Demand_7 hours future', 'Forecast_Demand_7.5 hours future', 'Forecast_Demand_8 hours future', 'Forecast_Demand_8.5 hours future', 'Forecast_Demand_9 hours future', 'Forecast_Demand_9.5 hours future', 'Forecast_Roof_0.5 hour future', 'Forecast_Roof_1 hour future', 'Forecast_Roof_1.5 hours future', 'Forecast_Roof_2 hours future', 'Forecast_Roof_2.5 hours future', 'Forecast_Roof_3 hours future', 'Forecast_Roof_3.5 hours future', 'Forecast_Roof_4 hours future', 'Forecast_Roof_4.5 hours future', 

In [48]:
big_df = pd.read_csv("merged_data.csv",parse_dates=["date"])
big_df

,date,Region,Forecast_Demand_0.5 hour future,Forecast_Demand_1 hour future,Forecast_Demand_1.5 hours future,Forecast_Demand_2 hours future,Forecast_Demand_2.5 hours future,Forecast_Demand_3 hours future,Forecast_Demand_3.5 hours future,Forecast_Demand_4 hours future,...,Forecast_Roof_7 hours future,Forecast_Roof_7.5 hours future,Forecast_Roof_8 hours future,Forecast_Roof_8.5 hours future,Forecast_Roof_9 hours future,Forecast_Roof_9.5 hours future,Forecast_Roof_10 hours future,Actual_Demand_OPERATIONAL_DEMAND,Actual_Roof_OPOWER,Actual_Roof_LASTCHANGED
0,2023-08-31 00:00:00,NSW1,7732.0,7594.0,7407.0,7187.0,6958.0,6768.0,6664.0,6628.0,...,83.215,392.035,833.851,1311.266,1765.236,2157.137,2490.814,7865.0,0.0,2023/08/31 00:19:07
1,2023-08-31 00:00:00,QLD1,5561.0,5436.0,5365.0,5306.0,5254.0,5236.0,5242.0,5264.0,...,201.800,591.712,1074.146,1564.232,2012.651,2415.906,2742.236,5749.0,0.0,2023/08/31 00:19:07
2,2023-08-31 00:00:00,SA1,1526.0,1537.0,1522.0,1483.0,1427.0,1378.0,1337.0,1311.0,...,0.000,0.166,41.919,135.681,254.413,372.708,482.970,1538.0,0.0,2023/08/31 00:19:07
3,2023-08-31 00:00:00,TAS1,1133.0,1123.0,1115.0,1109.0,1112.0,1109.0,1112.0,1110.0,...,0.044,4.879,17.656,35.796,55.752,75.996,95.247,1165.0,0.0,2023/08/31 00:19:07
4,2023-08-31 00:00:00,VIC1,5091.0,4942.0,4814.0,4678.0,4566.0,4487.0,4449.0,4444.0,...,0.000,35.756,176.805,366.140,568.436,755.113,902.785,5267.0,0.0,2023/08/31 00:19:07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94070,2024-09-25 23:00:00,NSW1,7455.0,7378.0,7260.0,7123.0,6957.0,6751.0,6582.0,6458.0,...,0.109,78.519,232.056,432.474,643.848,855.430,1066.236,7550.0,0.0,2024/09/25 23:19:12
94071,2024-09-25 23:00:00,QLD1,6052.0,5895.0,5752.0,5630.0,5541.0,5478.0,5425.0,5393.0,...,0.444,240.133,683.919,1217.739,1771.921,2276.854,2691.584,6185.0,0.0,2024/09/25 23:19:12
94072,2024-09-25 23:00:00,SA1,1573.0,1595.0,1581.0,1591.0,1564.0,1527.0,1480.0,1435.0,...,0.000,0.000,0.000,105.073,287.182,509.605,734.053,1632.0,0.0,2024/09/25 23:19:12
94073,2024-09-25 23:00:00,TAS1,1180.0,1161.0,1153.0,1143.0,1133.0,1128.0,1128.0,1132.0,...,0.024,0.102,13.059,35.706,61.367,87.326,109.547,1216.0,0.0,2024/09/25 23:19:12


In [49]:
import pandas as pd


def convert_30min_to_5min(df):
    # 確保日期列是datetime類型
    df['date'] = pd.to_datetime(df['date'])
    
    # 按日期和地區排序
    df = df.sort_values(['date', 'Region'])
    
    # 使用列表推導式創建新的5分鐘頻率的數據
    new_rows = [
        row.copy()
        for _, row in df.iterrows()
        for _ in range(6)
    ]
    
    # 使用concat創建新的DataFrame
    df_5min = pd.concat(new_rows, axis=1).T
    
    # 更新日期列
    df_5min['date'] = [
        original_date + pd.Timedelta(minutes=5*i)
        for original_date in df['date']
        for i in range(6)
    ]
    
    # 按日期和地區排序
    df_5min = df_5min.sort_values(['date', 'Region']).reset_index(drop=True)
    
    return df_5min



min5_df = convert_30min_to_5min(big_df)
min5_df

# 使用示例：
# new_dfs["Actual Roof"] = convert_30min_to_5min(new_dfs["Actual Roof"])

,date,Region,Forecast_Demand_0.5 hour future,Forecast_Demand_1 hour future,Forecast_Demand_1.5 hours future,Forecast_Demand_2 hours future,Forecast_Demand_2.5 hours future,Forecast_Demand_3 hours future,Forecast_Demand_3.5 hours future,Forecast_Demand_4 hours future,...,Forecast_Roof_7 hours future,Forecast_Roof_7.5 hours future,Forecast_Roof_8 hours future,Forecast_Roof_8.5 hours future,Forecast_Roof_9 hours future,Forecast_Roof_9.5 hours future,Forecast_Roof_10 hours future,Actual_Demand_OPERATIONAL_DEMAND,Actual_Roof_OPOWER,Actual_Roof_LASTCHANGED
0,2023-08-31 00:00:00,NSW1,7732.0,7594.0,7407.0,7187.0,6958.0,6768.0,6664.0,6628.0,...,83.215,392.035,833.851,1311.266,1765.236,2157.137,2490.814,7865.0,0.0,2023/08/31 00:19:07
1,2023-08-31 00:00:00,QLD1,5561.0,5436.0,5365.0,5306.0,5254.0,5236.0,5242.0,5264.0,...,201.8,591.712,1074.146,1564.232,2012.651,2415.906,2742.236,5749.0,0.0,2023/08/31 00:19:07
2,2023-08-31 00:00:00,SA1,1526.0,1537.0,1522.0,1483.0,1427.0,1378.0,1337.0,1311.0,...,0.0,0.166,41.919,135.681,254.413,372.708,482.97,1538.0,0.0,2023/08/31 00:19:07
3,2023-08-31 00:00:00,TAS1,1133.0,1123.0,1115.0,1109.0,1112.0,1109.0,1112.0,1110.0,...,0.044,4.879,17.656,35.796,55.752,75.996,95.247,1165.0,0.0,2023/08/31 00:19:07
4,2023-08-31 00:00:00,VIC1,5091.0,4942.0,4814.0,4678.0,4566.0,4487.0,4449.0,4444.0,...,0.0,35.756,176.805,366.14,568.436,755.113,902.785,5267.0,0.0,2023/08/31 00:19:07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564445,2024-09-25 23:25:00,NSW1,7455.0,7378.0,7260.0,7123.0,6957.0,6751.0,6582.0,6458.0,...,0.109,78.519,232.056,432.474,643.848,855.43,1066.236,7550.0,0.0,2024/09/25 23:19:12
564446,2024-09-25 23:25:00,QLD1,6052.0,5895.0,5752.0,5630.0,5541.0,5478.0,5425.0,5393.0,...,0.444,240.133,683.919,1217.739,1771.921,2276.854,2691.584,6185.0,0.0,2024/09/25 23:19:12
564447,2024-09-25 23:25:00,SA1,1573.0,1595.0,1581.0,1591.0,1564.0,1527.0,1480.0,1435.0,...,0.0,0.0,0.0,105.073,287.182,509.605,734.053,1632.0,0.0,2024/09/25 23:19:12
564448,2024-09-25 23:25:00,TAS1,1180.0,1161.0,1153.0,1143.0,1133.0,1128.0,1128.0,1132.0,...,0.024,0.102,13.059,35.706,61.367,87.326,109.547,1216.0,0.0,2024/09/25 23:19:12


In [50]:
min5_df

,date,Region,Forecast_Demand_0.5 hour future,Forecast_Demand_1 hour future,Forecast_Demand_1.5 hours future,Forecast_Demand_2 hours future,Forecast_Demand_2.5 hours future,Forecast_Demand_3 hours future,Forecast_Demand_3.5 hours future,Forecast_Demand_4 hours future,...,Forecast_Roof_7 hours future,Forecast_Roof_7.5 hours future,Forecast_Roof_8 hours future,Forecast_Roof_8.5 hours future,Forecast_Roof_9 hours future,Forecast_Roof_9.5 hours future,Forecast_Roof_10 hours future,Actual_Demand_OPERATIONAL_DEMAND,Actual_Roof_OPOWER,Actual_Roof_LASTCHANGED
0,2023-08-31 00:00:00,NSW1,7732.0,7594.0,7407.0,7187.0,6958.0,6768.0,6664.0,6628.0,...,83.215,392.035,833.851,1311.266,1765.236,2157.137,2490.814,7865.0,0.0,2023/08/31 00:19:07
1,2023-08-31 00:00:00,QLD1,5561.0,5436.0,5365.0,5306.0,5254.0,5236.0,5242.0,5264.0,...,201.8,591.712,1074.146,1564.232,2012.651,2415.906,2742.236,5749.0,0.0,2023/08/31 00:19:07
2,2023-08-31 00:00:00,SA1,1526.0,1537.0,1522.0,1483.0,1427.0,1378.0,1337.0,1311.0,...,0.0,0.166,41.919,135.681,254.413,372.708,482.97,1538.0,0.0,2023/08/31 00:19:07
3,2023-08-31 00:00:00,TAS1,1133.0,1123.0,1115.0,1109.0,1112.0,1109.0,1112.0,1110.0,...,0.044,4.879,17.656,35.796,55.752,75.996,95.247,1165.0,0.0,2023/08/31 00:19:07
4,2023-08-31 00:00:00,VIC1,5091.0,4942.0,4814.0,4678.0,4566.0,4487.0,4449.0,4444.0,...,0.0,35.756,176.805,366.14,568.436,755.113,902.785,5267.0,0.0,2023/08/31 00:19:07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564445,2024-09-25 23:25:00,NSW1,7455.0,7378.0,7260.0,7123.0,6957.0,6751.0,6582.0,6458.0,...,0.109,78.519,232.056,432.474,643.848,855.43,1066.236,7550.0,0.0,2024/09/25 23:19:12
564446,2024-09-25 23:25:00,QLD1,6052.0,5895.0,5752.0,5630.0,5541.0,5478.0,5425.0,5393.0,...,0.444,240.133,683.919,1217.739,1771.921,2276.854,2691.584,6185.0,0.0,2024/09/25 23:19:12
564447,2024-09-25 23:25:00,SA1,1573.0,1595.0,1581.0,1591.0,1564.0,1527.0,1480.0,1435.0,...,0.0,0.0,0.0,105.073,287.182,509.605,734.053,1632.0,0.0,2024/09/25 23:19:12
564448,2024-09-25 23:25:00,TAS1,1180.0,1161.0,1153.0,1143.0,1133.0,1128.0,1128.0,1132.0,...,0.024,0.102,13.059,35.706,61.367,87.326,109.547,1216.0,0.0,2024/09/25 23:19:12


In [51]:
min5_df.to_csv("5minbig.csv",index=False)

In [52]:
newmin5_df = pd.read_csv("5minbig.csv",parse_dates=["date"])

In [53]:
QLD_df = newmin5_df[newmin5_df["Region"]=="QLD1"]
QLD_df.shape

(112890, 44)

In [54]:
print(QLD_df.head())
print(QLD_df.tail())

                  date Region  Forecast_Demand_0.5 hour future  \
1  2023-08-31 00:00:00   QLD1                           5561.0   
6  2023-08-31 00:05:00   QLD1                           5561.0   
11 2023-08-31 00:10:00   QLD1                           5561.0   
16 2023-08-31 00:15:00   QLD1                           5561.0   
21 2023-08-31 00:20:00   QLD1                           5561.0   

    Forecast_Demand_1 hour future  Forecast_Demand_1.5 hours future  \
1                          5436.0                            5365.0   
6                          5436.0                            5365.0   
11                         5436.0                            5365.0   
16                         5436.0                            5365.0   
21                         5436.0                            5365.0   

    Forecast_Demand_2 hours future  Forecast_Demand_2.5 hours future  \
1                           5306.0                            5254.0   
6                           5306

In [55]:
QLD_Train = pd.read_csv("./Trim/Train.csv",parse_dates=["date"])
merged_df = pd.merge(QLD_Train, QLD_df, on="date", how='left')

In [56]:
merged_df

,date,Demand,Price,Region,Forecast_Demand_0.5 hour future,Forecast_Demand_1 hour future,Forecast_Demand_1.5 hours future,Forecast_Demand_2 hours future,Forecast_Demand_2.5 hours future,Forecast_Demand_3 hours future,...,Forecast_Roof_7 hours future,Forecast_Roof_7.5 hours future,Forecast_Roof_8 hours future,Forecast_Roof_8.5 hours future,Forecast_Roof_9 hours future,Forecast_Roof_9.5 hours future,Forecast_Roof_10 hours future,Actual_Demand_OPERATIONAL_DEMAND,Actual_Roof_OPOWER,Actual_Roof_LASTCHANGED
0,2023-08-31 00:00:00,5576.36,71.67,QLD1,5561.0,5436.0,5365.0,5306.0,5254.0,5236.0,...,201.800,591.712,1074.146,1564.232,2012.651,2415.906,2742.236,5749.0,0.0,2023/08/31 00:19:07
1,2023-08-31 00:05:00,5609.59,72.19,QLD1,5561.0,5436.0,5365.0,5306.0,5254.0,5236.0,...,201.800,591.712,1074.146,1564.232,2012.651,2415.906,2742.236,5749.0,0.0,2023/08/31 00:19:07
2,2023-08-31 00:10:00,5606.26,72.77,QLD1,5561.0,5436.0,5365.0,5306.0,5254.0,5236.0,...,201.800,591.712,1074.146,1564.232,2012.651,2415.906,2742.236,5749.0,0.0,2023/08/31 00:19:07
3,2023-08-31 00:15:00,5481.22,69.10,QLD1,5561.0,5436.0,5365.0,5306.0,5254.0,5236.0,...,201.800,591.712,1074.146,1564.232,2012.651,2415.906,2742.236,5749.0,0.0,2023/08/31 00:19:07
4,2023-08-31 00:20:00,5463.24,69.10,QLD1,5561.0,5436.0,5365.0,5306.0,5254.0,5236.0,...,201.800,591.712,1074.146,1564.232,2012.651,2415.906,2742.236,5749.0,0.0,2023/08/31 00:19:07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112880,2024-09-25 22:40:00,6141.45,93.83,QLD1,6160.0,6027.0,5892.0,5753.0,5622.0,5532.0,...,0.000,0.146,240.195,683.743,1217.094,1771.738,2275.414,6354.0,0.0,2024/09/25 22:49:13
112881,2024-09-25 22:45:00,6216.81,93.83,QLD1,6160.0,6027.0,5892.0,5753.0,5622.0,5532.0,...,0.000,0.146,240.195,683.743,1217.094,1771.738,2275.414,6354.0,0.0,2024/09/25 22:49:13
112882,2024-09-25 22:50:00,6209.61,96.95,QLD1,6160.0,6027.0,5892.0,5753.0,5622.0,5532.0,...,0.000,0.146,240.195,683.743,1217.094,1771.738,2275.414,6354.0,0.0,2024/09/25 22:49:13
112883,2024-09-25 22:55:00,6137.80,89.00,QLD1,6160.0,6027.0,5892.0,5753.0,5622.0,5532.0,...,0.000,0.146,240.195,683.743,1217.094,1771.738,2275.414,6354.0,0.0,2024/09/25 22:49:13


In [57]:
import pandas as pd

def adjust_opower_values(df):
    # 確保日期列和 LASTCHANGED 列是 datetime 類型
    df['date'] = pd.to_datetime(df['date'])
    df['Actual_Roof_LASTCHANGED'] = pd.to_datetime(df['Actual_Roof_LASTCHANGED'])
    
    # 按日期排序
    df = df.sort_values('date')
    
    # 創建一個新的列來存儲調整後的 OPOWER 值
    df['Adjusted_Actual_Roof_OPOWER'] = 0.0
    
    last_valid_opower = 0.0
    last_valid_time = pd.Timestamp.min
    
    for index, row in df.iterrows():
        if row['date'] > row['Actual_Roof_LASTCHANGED'] and row['Actual_Roof_LASTCHANGED'] > last_valid_time:
            last_valid_opower = row['Actual_Roof_OPOWER']
            last_valid_time = row['Actual_Roof_LASTCHANGED']
        
        df.at[index, 'Adjusted_Actual_Roof_OPOWER'] = last_valid_opower
    
    return df
merged_df = adjust_opower_values(merged_df)
merged_df
# 使用示例：
# new_dfs["Actual Roof"] = adjust_opower_values(new_dfs["Actual Roof"])

,date,Demand,Price,Region,Forecast_Demand_0.5 hour future,Forecast_Demand_1 hour future,Forecast_Demand_1.5 hours future,Forecast_Demand_2 hours future,Forecast_Demand_2.5 hours future,Forecast_Demand_3 hours future,...,Forecast_Roof_7.5 hours future,Forecast_Roof_8 hours future,Forecast_Roof_8.5 hours future,Forecast_Roof_9 hours future,Forecast_Roof_9.5 hours future,Forecast_Roof_10 hours future,Actual_Demand_OPERATIONAL_DEMAND,Actual_Roof_OPOWER,Actual_Roof_LASTCHANGED,Adjusted_Actual_Roof_OPOWER
0,2023-08-31 00:00:00,5576.36,71.67,QLD1,5561.0,5436.0,5365.0,5306.0,5254.0,5236.0,...,591.712,1074.146,1564.232,2012.651,2415.906,2742.236,5749.0,0.0,2023-08-31 00:19:07,0.0
1,2023-08-31 00:05:00,5609.59,72.19,QLD1,5561.0,5436.0,5365.0,5306.0,5254.0,5236.0,...,591.712,1074.146,1564.232,2012.651,2415.906,2742.236,5749.0,0.0,2023-08-31 00:19:07,0.0
2,2023-08-31 00:10:00,5606.26,72.77,QLD1,5561.0,5436.0,5365.0,5306.0,5254.0,5236.0,...,591.712,1074.146,1564.232,2012.651,2415.906,2742.236,5749.0,0.0,2023-08-31 00:19:07,0.0
3,2023-08-31 00:15:00,5481.22,69.10,QLD1,5561.0,5436.0,5365.0,5306.0,5254.0,5236.0,...,591.712,1074.146,1564.232,2012.651,2415.906,2742.236,5749.0,0.0,2023-08-31 00:19:07,0.0
4,2023-08-31 00:20:00,5463.24,69.10,QLD1,5561.0,5436.0,5365.0,5306.0,5254.0,5236.0,...,591.712,1074.146,1564.232,2012.651,2415.906,2742.236,5749.0,0.0,2023-08-31 00:19:07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112880,2024-09-25 22:40:00,6141.45,93.83,QLD1,6160.0,6027.0,5892.0,5753.0,5622.0,5532.0,...,0.146,240.195,683.743,1217.094,1771.738,2275.414,6354.0,0.0,2024-09-25 22:49:13,0.0
112881,2024-09-25 22:45:00,6216.81,93.83,QLD1,6160.0,6027.0,5892.0,5753.0,5622.0,5532.0,...,0.146,240.195,683.743,1217.094,1771.738,2275.414,6354.0,0.0,2024-09-25 22:49:13,0.0
112882,2024-09-25 22:50:00,6209.61,96.95,QLD1,6160.0,6027.0,5892.0,5753.0,5622.0,5532.0,...,0.146,240.195,683.743,1217.094,1771.738,2275.414,6354.0,0.0,2024-09-25 22:49:13,0.0
112883,2024-09-25 22:55:00,6137.80,89.00,QLD1,6160.0,6027.0,5892.0,5753.0,5622.0,5532.0,...,0.146,240.195,683.743,1217.094,1771.738,2275.414,6354.0,0.0,2024-09-25 22:49:13,0.0


In [58]:
merged_df.to_csv("final_train.csv",index=False)